## Model Training

In [ ]:
import torch
import numpy

from torch.utils.data import DataLoader

from src.data.transforms import Transforms
from src.data.dataset import SkinDataset
from src.model.cnn import SkinClassifierNet
from src.model.train import ModelTraining
from src.utils.config import Config

In [ ]:
numpy.random.seed(1234)
torch.random.manual_seed(1324)

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

IMAGES_PATH = Config.get_path('images')
TRAINING_PATH = Config.get_path('training_csv')
MODEL_PATH = Config.get_path('model')

BATCH_SIZE = Config.get_setting_int('batch_size')
NUM_EPOCHS = Config.get_setting_int('num_epochs')
NUM_WORKERS = Config.get_setting_int('num_workers')

LEARNING_RATE = Config.get_setting_float('learning_rate')
MOMENTUM = Config.get_setting_float('momentum')

In [ ]:
load_and_augment = Transforms.get_data_and_augment()

trainset = SkinDataset(
    IMAGES_PATH,
    TRAINING_PATH,
    load_and_augment
)
train_loader = DataLoader(
    trainset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

model = SkinClassifierNet()
criterion = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor([1.0, 2.0]))
optimizer = torch.optim.SGD(model.parameters(), LEARNING_RATE, MOMENTUM)

trainer = ModelTraining(DEVICE, model, MODEL_PATH)
trainer.fit(criterion, optimizer, NUM_EPOCHS, train_loader)
trainer.save()